In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile4 = pd.read_pickle("BBB_Train_Decile4.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile4.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile4, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 4')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 62/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4375 - acc: 0.8154 - val_loss: 0.4148 - val_acc: 0.8238
Epoch 63/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4404 - acc: 0.8118 - val_loss: 0.4147 - val_acc: 0.8238
Epoch 64/75
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4388 - acc: 0.8095 - val_loss: 0.4147 - val_acc: 0.8238
Epoch 65/75
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4403 - acc: 0.8128 - val_loss: 0.4151 - val_acc: 0.8251
Epoch 66/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4478 - acc: 0.8105 - val_loss: 0.4146 - val_acc: 0.8238
Epoch 67/75
3087/3087 [==============================] - 11s 3ms/step - loss: 0.4408 - acc: 0.8086 - val_loss: 0.4146 - val_acc: 0.8238
Epoch 68/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4441 - acc: 0.8082 - val_loss: 0.4147 - val_acc: 0.8238
Epoch 69/75
3087/3087 [=============================

3087/3087 [==============================] - 10s 3ms/step - loss: 0.4195 - acc: 0.8192 - val_loss: 0.4141 - val_acc: 0.8225
Epoch 46/75
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4122 - acc: 0.8199 - val_loss: 0.4142 - val_acc: 0.8238
Epoch 47/75
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4195 - acc: 0.8212 - val_loss: 0.4142 - val_acc: 0.8238
Epoch 48/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4161 - acc: 0.8189 - val_loss: 0.4142 - val_acc: 0.8225
Epoch 49/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4164 - acc: 0.8144 - val_loss: 0.4142 - val_acc: 0.8251
Epoch 50/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4158 - acc: 0.8238 - val_loss: 0.4142 - val_acc: 0.8225
Epoch 51/75
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4126 - acc: 0.8247 - val_loss: 0.4142 - val_acc: 0.8251
Epoch 52/75
3087/3087 [==============================] - 10s 3m

3087/3087 [==============================] - 10s 3ms/step - loss: 0.4324 - acc: 0.8134 - val_loss: 0.4163 - val_acc: 0.8212
Epoch 29/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4308 - acc: 0.8147 - val_loss: 0.4159 - val_acc: 0.8199
Epoch 30/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4291 - acc: 0.8179 - val_loss: 0.4158 - val_acc: 0.8199
Epoch 31/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4335 - acc: 0.8199 - val_loss: 0.4160 - val_acc: 0.8225
Epoch 32/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4385 - acc: 0.8160 - val_loss: 0.4159 - val_acc: 0.8225
Epoch 33/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4274 - acc: 0.8173 - val_loss: 0.4159 - val_acc: 0.8225
Epoch 34/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4241 - acc: 0.8150 - val_loss: 0.4160 - val_acc: 0.8225
Epoch 35/100
3087/3087 [==============================]

3087/3087 [==============================] - 8s 3ms/step - loss: 0.4257 - acc: 0.8192 - val_loss: 0.4094 - val_acc: 0.8225
Epoch 40/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4303 - acc: 0.8186 - val_loss: 0.4095 - val_acc: 0.8238
Epoch 41/75
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4316 - acc: 0.8196 - val_loss: 0.4096 - val_acc: 0.8238
Epoch 42/75
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4278 - acc: 0.8173 - val_loss: 0.4095 - val_acc: 0.8238
Epoch 43/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4235 - acc: 0.8137 - val_loss: 0.4095 - val_acc: 0.8225
Epoch 44/75
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4234 - acc: 0.8173 - val_loss: 0.4094 - val_acc: 0.8212
Epoch 45/75
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4220 - acc: 0.8173 - val_loss: 0.4097 - val_acc: 0.8212
Epoch 46/75
3087/3087 [==============================] - 9s 3ms/ste

...

3087/3087 [==============================] - 9s 3ms/step - loss: 0.4390 - acc: 0.8115 - val_loss: 0.4128 - val_acc: 0.8238
Epoch 91/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4399 - acc: 0.8076 - val_loss: 0.4129 - val_acc: 0.8251
Epoch 92/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4369 - acc: 0.8128 - val_loss: 0.4127 - val_acc: 0.8212
Epoch 93/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4425 - acc: 0.8128 - val_loss: 0.4129 - val_acc: 0.8251
Epoch 94/100
3087/3087 [==============================] - 10s 3ms/step - loss: 0.4444 - acc: 0.8124 - val_loss: 0.4128 - val_acc: 0.8251
Epoch 95/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4421 - acc: 0.8163 - val_loss: 0.4126 - val_acc: 0.8212
Epoch 96/100
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4205 - acc: 0.8128 - val_loss: 0.4123 - val_acc: 0.8161
Epoch 21/50
3087/3087 [==============================] - 9s 

Epoch 32/125
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4577 - acc: 0.8105 - val_loss: 0.4391 - val_acc: 0.8109
Epoch 33/125
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4603 - acc: 0.8047 - val_loss: 0.4402 - val_acc: 0.8096
Epoch 34/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4572 - acc: 0.8121 - val_loss: 0.4390 - val_acc: 0.8096
Epoch 35/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4582 - acc: 0.8073 - val_loss: 0.4367 - val_acc: 0.8135
Epoch 36/125
3087/3087 [==============================] - 8s 2ms/step - loss: 0.4520 - acc: 0.8079 - val_loss: 0.4362 - val_acc: 0.8148
Epoch 37/125
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4515 - acc: 0.8102 - val_loss: 0.4352 - val_acc: 0.8148
Epoch 38/125
3087/3087 [==============================] - 9s 3ms/step - loss: 0.4528 - acc: 0.8134 - val_loss: 0.4375 - val_acc: 0.8174
Epoch 39/125
3087/3087 [========================

3087/3087 [==============================] - 6s 2ms/step - loss: 0.4615 - acc: 0.7975 - val_loss: 0.4289 - val_acc: 0.8225
Epoch 7/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4489 - acc: 0.8063 - val_loss: 0.4261 - val_acc: 0.8174
Epoch 8/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4481 - acc: 0.8092 - val_loss: 0.4267 - val_acc: 0.8212
Epoch 9/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4445 - acc: 0.8111 - val_loss: 0.4217 - val_acc: 0.8174
Epoch 10/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4428 - acc: 0.8050 - val_loss: 0.4232 - val_acc: 0.8225
Epoch 11/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4390 - acc: 0.8092 - val_loss: 0.4208 - val_acc: 0.8225
Epoch 12/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4388 - acc: 0.8121 - val_loss: 0.4241 - val_acc: 0.8161
Epoch 13/100
3087/3087 [==============================] - 6s 2ms

3087/3087 [==============================] - 3s 1ms/step - loss: 0.4236 - acc: 0.8192 - val_loss: 0.4137 - val_acc: 0.8148
Epoch 65/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4335 - acc: 0.8124 - val_loss: 0.4136 - val_acc: 0.8148
Epoch 66/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4203 - acc: 0.8212 - val_loss: 0.4136 - val_acc: 0.8148
Epoch 67/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4202 - acc: 0.8202 - val_loss: 0.4137 - val_acc: 0.8148
Epoch 68/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4222 - acc: 0.8231 - val_loss: 0.4139 - val_acc: 0.8161
Epoch 69/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4258 - acc: 0.8196 - val_loss: 0.4138 - val_acc: 0.8148
Epoch 70/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4220 - acc: 0.8205 - val_loss: 0.4138 - val_acc: 0.8148
Epoch 71/100
3087/3087 [==============================] - 3s 

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4239 - acc: 0.8134 - val_loss: 0.4150 - val_acc: 0.8225
Epoch 24/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4327 - acc: 0.8115 - val_loss: 0.4143 - val_acc: 0.8225
Epoch 25/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4335 - acc: 0.8179 - val_loss: 0.4144 - val_acc: 0.8212
Epoch 26/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4261 - acc: 0.8144 - val_loss: 0.4139 - val_acc: 0.8225
Epoch 27/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4272 - acc: 0.8199 - val_loss: 0.4141 - val_acc: 0.8225
Epoch 28/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4298 - acc: 0.8141 - val_loss: 0.4131 - val_acc: 0.8225
Epoch 29/50
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4301 - acc: 0.8137 - val_loss: 0.4134 - val_acc: 0.8225
Epoch 30/50
3087/3087 [==============================] - 4s 1ms/ste

In [ ]:
0.8297